In [ ]:
import torch
import open3d.ml.torch as ml3d
import pickle

# use roco4 for this notebook

with open('outputs.pkl', 'rb') as f:
    outputs = pickle.load(f)

# clip stuff
# pixelwise_features = torch.load("clip_features.pt").float().cpu()
# xyz_pts = torch.load("clip_xyz.pt").float().cpu()

# blip stuff
pixelwise_features = torch.load("blip_noneva_pixelwise_rendfet_features.pt").float().cpu()
xyz_pts = torch.tensor(outputs['xyz_pts']).float().cpu()


# sanity check
print(pixelwise_features.shape)
print(xyz_pts.shape)
print(pixelwise_features[1000])
print(xyz_pts[1000])

- First, since we initialized the feature matrix as a matrix of zeros, we remove any rows that didn't get assigned to a class so they don't impact the representation of any other points.

In [ ]:
from time import time

zero_time = time()
zero_row_indices = torch.all(pixelwise_features == 0, dim=1)
pixelwise_features = pixelwise_features[~zero_row_indices]
xyz_pts = xyz_pts[~zero_row_indices.cpu().numpy()]
print("Time taken to remove zero rows: ", time() - zero_time)

# sanity check
print(pixelwise_features.shape)
print(xyz_pts.shape)
print(pixelwise_features[1000])
print(xyz_pts[1000])

In [ ]:
i = 0
agg_rate = 0.1
pts_remaining = [xyz_pts.shape[0]]
while xyz_pts.shape[0] > 20000:
    xyz_pts, pixelwise_features = ml3d.ops.voxel_pooling(xyz_pts, pixelwise_features, agg_rate, position_fn='nearest_neighbor', feature_fn='nearest_neighbor')
    i += 1

    agg_rate += 0.1
    pts_remaining.append(xyz_pts.shape[0])

print(f"Completed voxel aggregation with {xyz_pts.shape[0]} points")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(pts_remaining)
plt.xlabel("Voxel aggregation iteration")
plt.ylabel("Number of points remaining")
plt.title("Voxel aggregation iterations vs. number of points remaining")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import open3d as o3d

fig = plt.figure(figsize=(6, 6), dpi=160)
ax = fig.add_subplot(111, projection="3d")
background_color = (1.0, 1.0, 1.0)
ax.set_facecolor(background_color)
ax.w_xaxis.set_pane_color(background_color)  # type: ignore
ax.w_yaxis.set_pane_color(background_color)  # type: ignore
ax.w_zaxis.set_pane_color(background_color)  # type: ignore

x, y, z = (
    xyz_pts[:, 0],
    xyz_pts[:, 1],
    xyz_pts[:, 2],
)
ax.scatter(
    x,
    y,
    z,
)
plt.tight_layout(pad=0)
# plt.show()

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(xyz_pts)

# Save the point cloud to a .ply file
o3d.io.write_point_cloud("rotated_point_cloud_aggregated.ply", point_cloud)

In [ ]:
# now we normalize the features for the image as a whole
pixelwise_features = torch.nn.functional.normalize(pixelwise_features, dim=-1)

In [ ]:
torch.save(pixelwise_features, "blip_noneva_pixelwise_rendfet_features_reduced.pt")
torch.save(xyz_pts, "blip_noneva_pixelwise_rendfet_xyz_reduced.pt")